## Scikit-Learn Logistic Regression

The following example will:

* Set the input and output schemas.
* Upload a SKLearn Logistic Regression model to Wallaroo.
* Deploy a pipeline with the uploaded SKLearn model as a pipeline step.
* Perform a test inference.
* Undeploy the pipeline.

In [2]:
import json
import os
import pickle

import wallaroo
from wallaroo.pipeline   import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework

import pyarrow as pa
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression



In [3]:
# wl = wallaroo.Client()

wallarooPrefix = ""
wallarooSuffix = "autoscale-uat-ee.wallaroo.dev"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}keycloak.{wallarooSuffix}", 
                    auth_type="sso")

Please log into the following URL in a web browser:

	https://keycloak.autoscale-uat-ee.wallaroo.dev/auth/realms/master/device?user_code=PHQF-IMWH

Login successful!


In [4]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

prefix = "sklearn-kmeans"

In [5]:
workspace = get_workspace(f"{prefix}-jch")
wl.set_current_workspace(workspace)

{'name': 'sklearn-kmeans-jch', 'id': 34, 'archived': False, 'created_by': 'd9a72bd9-2a1c-44dd-989f-3c7c15130885', 'created_at': '2023-06-27T22:01:20.995281+00:00', 'models': [], 'pipelines': []}

## Data & Model Creation

In [6]:
input_schema = pa.schema([
    pa.field('sepal length (cm)', pa.float64()),
    pa.field('sepal width (cm)', pa.float64()),
    pa.field('petal length (cm)', pa.float64()),
    pa.field('petal width (cm)', pa.float64())
])

output_schema = pa.schema([
    pa.field('output', pa.float64())
])

## Upload model

In [10]:
model = wl.upload_model(f"{prefix}", 
                        'models/model-auto-conversion_sklearn_kmeans.pkl', 
                        framework=Framework.SKLEARN, 
                        input_schema=input_schema, 
                        output_schema=output_schema)
model

{'name': 'sklearn-kmeans', 'version': '12863f01-4738-44d6-b178-1c22805300b1', 'file_name': 'model-auto-conversion_sklearn_kmeans.pkl', 'image_path': None, 'last_update_time': datetime.datetime(2023, 6, 27, 22, 7, 55, 639896, tzinfo=tzutc())}

## Configure model and pipeline

In [8]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [9]:
pipeline_name = f"{prefix}-pipeline"
pipeline = wl.build_pipeline(pipeline_name)
pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

Exception: Failed to determine model configuration, could not auto configure based on name, and no existing model configuration was present. 

## Inference

In [ ]:
pipeline.infer_from_file('data/test-sklearn-kmeans.json')

In [ ]:
pipeline.undeploy()